In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from itertools import combinations
import statsmodels.api as sm
import time

In [2]:
def import_data(experimental_file, response_file, response):
    'imports data'
    'experimental_file = experimental design csv filename i.e experimental.csv'
    'response_file = results csv filename i.e Response.csv'
    'response = reponse name: i.e rheomix final deg time min or rheomix stability time min'
    
    experimental_df = pd.read_csv(experimental_file)
    response_df = pd.read_csv(response_file)
    
    X = experimental_df[experimental_df.columns.values.tolist()[1:]].values
    y = response_df[response].values
    max1 = max(y)
    min1 = min(y)

    y_norm = [2*((i-min1)/(max1-min1)) - 1 for i in y]
    
    
    X_linear = X
    linear_terms = experimental_df.columns.values.tolist()[1:]
    
    return y_norm, X_linear, linear_terms, experimental_df, response_df
    

In [3]:
def X_gen(model, X_linear):

    for i, j in enumerate(model):
        
        if i == 0 and len(j) == 2: 
            X_new = X_linear[:, model[0][1]]
        
        else:
            
            if len(j) == 2:
                add = X_linear[:, j[1]]

            if len(j) == 3:
                add = X_linear[:, j[1]]*X_linear[:, j[2]]

            X_new2 = np.column_stack((X_new, add))
            X_new = X_new2

    
    return X_new

In [4]:
start = time.time()


# linear_terms = experimental_df.columns.values.tolist()[1:]



def fit_model(experimental_file, response_file, response):

    y, X_linear, linear_terms, experimental_df, response_df = import_data(experimental_file, response_file, response)
    lin_terms = []
    AIC_prev = 1000

    for i in range(len(linear_terms)):
        term = linear_terms[i]
        key = i
        lin_terms.append([term, i])

    cnt = 0
    for i in range(len(lin_terms)):
        for j in combinations(lin_terms, i+1):
            linear_terms = list(j)


            model = [a for a in linear_terms]
            cnt+=1

            X = X_gen(model, X_linear)

            model_fit = sm.OLS(y, X)
            results = model_fit.fit()
            AIC_cur = results.aic


            if AIC_cur < AIC_prev:

                AIC_prev = AIC_cur
                final_model = [model, results, AIC_cur]




            poss_terms = []
            for i in range(len(linear_terms)):
                for j in range(len(linear_terms)): 
                    if i < j:
                        poss_terms.append([linear_terms[i][0] + '*' + linear_terms[j][0],  linear_terms[i][1], linear_terms[j][1]])



            for m in range(1, len(poss_terms) + 1):
                for k in combinations(poss_terms, m):

                    model = [a for a in linear_terms]
                    for i in range(m):
                        model.append(k[i])

                    cnt +=1
                    X = X_gen(model, X_linear)

                    model_fit = sm.OLS(y, X)
                    results = model_fit.fit()
                    AIC_cur = results.aic



                    if AIC_cur < AIC_prev:

                        AIC_prev = AIC_cur
                        final_model = [model, results, AIC_cur]




    return final_model[0], final_model[1]


In [5]:
experimental_file = 'experimental.csv'
response_file = 'Response.csv'


test1 = 'rheomix final deg time min'
test2 = 'rheomix stability time min'
response = test2


model, results = fit_model(experimental_file, response_file, response)


In [6]:
model, results.summary()

([['Xpvc', 0],
  ['Xfiller', 1],
  ['Xstabiliser', 3],
  ['Xdinp', 4],
  ['Xldh', 5],
  ['Xfiller*Xldh', 1, 5],
  ['Xstabiliser*Xldh', 3, 5],
  ['Xdinp*Xldh', 4, 5]],
 <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      y   R-squared:                       0.888
 Model:                            OLS   Adj. R-squared:                  0.864
 Method:                 Least Squares   F-statistic:                     37.62
 Date:                Wed, 11 Sep 2019   Prob (F-statistic):           9.63e-16
 Time:                        19:56:02   Log-Likelihood:                 1.2433
 No. Observations:                  46   AIC:                             13.51
 Df Residuals:                      38   BIC:                             28.14
 Df Model:                           8                                         
 Covariance Type:            nonrobust                         